# Demonstration of Long vs. Wide Format

First we read in a table in the wide format.  The table becomes a `pandas` dataframe.

In [10]:
import pandas as pd
column_names = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year', 'origin', 'car_name']
auto = pd.read_table('../../data/auto-mpg.csv', sep = "\s+", header = None, names = column_names)
auto

<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\s'
/var/folders/_l/9c97q4ms35gg7r1hrlmv00cr0000ng/T/ipykernel_9462/2431901848.py:3: SyntaxWarning: invalid escape sequence '\s'
  auto = pd.read_table('../../data/auto-mpg.csv', sep = "\s+", header = None, names = column_names)


,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.00,2790.0,15.6,82,1,ford mustang gl
394,44.0,4,97.0,52.00,2130.0,24.6,82,2,vw pickup
395,32.0,4,135.0,84.00,2295.0,11.6,82,1,dodge rampage
396,28.0,4,120.0,79.00,2625.0,18.6,82,1,ford ranger


## Approach 1
Melt the table without considering a unique row id

In [11]:
LongFormWithoutEntities = pd.melt(auto, value_vars=column_names, var_name='Attribute', value_name='Value')
LongFormWithoutEntities

,Attribute,Value
0,mpg,18.0
1,mpg,15.0
2,mpg,18.0
3,mpg,16.0
4,mpg,17.0
...,...,...
3577,car_name,ford mustang gl
3578,car_name,vw pickup
3579,car_name,dodge rampage
3580,car_name,ford ranger


Every value in the original table has its own row in this table.  (398 * 9 = 3582)

The above table does not have enough information to reconstitute a wide table.  We do not have a way to find out which group of attribute values belong together (except if we consider row order).   

## Approach 2
Melt the table using car_name as our id

In [12]:
LongFormByCarName = pd.melt(auto, id_vars=['car_name'], var_name='Attribute', value_name='Value')
LongFormByCarName

,car_name,Attribute,Value
0,chevrolet chevelle malibu,mpg,18.0
1,buick skylark 320,mpg,15.0
2,plymouth satellite,mpg,18.0
3,amc rebel sst,mpg,16.0
4,ford torino,mpg,17.0
...,...,...,...
3179,ford mustang gl,origin,1
3180,vw pickup,origin,2
3181,dodge rampage,origin,1
3182,ford ranger,origin,1


Note that we have fewer rows in the long format.

In [13]:
WideFormUniqueCarName = pd.pivot_table(LongFormByCarName, index="car_name", columns="Attribute", values="Value", aggfunc='max')
WideFormUniqueCarName[column_names[:-1]] # order the columns the way they were in the orinal table

Attribute,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin
car_name,,,,,,,,
amc ambassador brougham,13.0,8,360.0,175.0,3821.0,11.0,73,1
amc ambassador dpl,15.0,8,390.0,190.0,3850.0,8.5,70,1
amc ambassador sst,17.0,8,304.0,150.0,3672.0,11.5,72,1
amc concord,24.3,6,232.0,90.00,3210.0,20.1,80,1
amc concord d/l,18.1,6,258.0,120.0,3410.0,15.1,78,1
...,...,...,...,...,...,...,...,...
vw dasher (diesel),43.4,4,90.0,48.00,2335.0,23.7,80,2
vw pickup,44.0,4,97.0,52.00,2130.0,24.6,82,2
vw rabbit,41.5,4,98.0,76.00,2144.0,14.7,80,2


In the last example, we lost some rows.  Presumably, 93 rows were for car names that are already listed among the 305 car names above.  These rows represent the same model but a different model year.  The values in these replicated rows get combined or aggregated into single values.  Our aggregation function was `max` which might not be appropriate.   Also the car names are now the row indices, which may or may not be good.

## Approach 3

In our next approach, we create a column that is unique for each table row.  The row indices are unique.  We can assign the values in the row index to this new column.   

In [14]:
auto["index"] = auto.index
auto

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name,index
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu,0
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320,1
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite,2
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst,3
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino,4
...,...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.00,2790.0,15.6,82,1,ford mustang gl,393
394,44.0,4,97.0,52.00,2130.0,24.6,82,2,vw pickup,394
395,32.0,4,135.0,84.00,2295.0,11.6,82,1,dodge rampage,395
396,28.0,4,120.0,79.00,2625.0,18.6,82,1,ford ranger,396


In [15]:
LongFormWithUniqueID = pd.melt(auto, id_vars="index", var_name='Attribute', value_name='Value')
LongFormWithUniqueID

,index,Attribute,Value
0,0,mpg,18.0
1,1,mpg,15.0
2,2,mpg,18.0
3,3,mpg,16.0
4,4,mpg,17.0
...,...,...,...
3577,393,car_name,ford mustang gl
3578,394,car_name,vw pickup
3579,395,car_name,dodge rampage
3580,396,car_name,ford ranger


Every value in the original table has its own row in this long format table.  

In [16]:
WideAgain = pd.pivot_table(LongFormWithUniqueID, index="index", columns="Attribute", values="Value", aggfunc='max')
WideAgain[column_names] # order the columns the way they were in the orinal table

Attribute,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name
index,,,,,,,,,
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.00,2790.0,15.6,82,1,ford mustang gl
394,44.0,4,97.0,52.00,2130.0,24.6,82,2,vw pickup
395,32.0,4,135.0,84.00,2295.0,11.6,82,1,dodge rampage


The last table is (almost) exactly like the original table.